# **Data** 📍

Within this section we will be importing the data, opening and reading it, and organizing the data into training and validation data.

In [ ]:
!wget --no-check-certificate \
    https://ceb.nlm.nih.gov/proj/malaria/cell_images.zip \
    -O /tmp/cell_images.zip

In [ ]:
import os
import zipfile

local_zip = '/tmp/cell_images.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
os.mkdir("/tmp/cell_images/train")
os.mkdir("/tmp/cell_images/valid")

In [ ]:
os.mkdir("/tmp/cell_images/train/Parasitized")
os.mkdir("/tmp/cell_images/train/Uninfected")

In [ ]:
import random
import shutil
source1 = '/tmp/cell_images/Parasitized'
dest1 = '/tmp/cell_images/train/Parasitized'
files = os.listdir(source1)
for fileName in random.sample(files, min(len(files), 10334)):
    path = os.path.join(source1, fileName)
    shutil.move(path, dest1)

In [ ]:
source2 = '/tmp/cell_images/Uninfected'
dest2 = '/tmp/cell_images/train/Uninfected'
files = os.listdir(source2)
for fileName in random.sample(files, min(len(files), 10334)):
    path = os.path.join(source2, fileName)
    shutil.move(path, dest2)

In [ ]:
parasite = '/tmp/cell_images/Parasitized'
uninfec = '/tmp/cell_images/Uninfected'
valid = '/tmp/cell_images/valid'
shutil.move(parasite,valid)
shutil.move(uninfec, valid)

In [ ]:
base_dir = '/tmp/cell_images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
train_Parasitized_dir = os.path.join(train_dir, 'Parasitized')
train_Uninfected_dir = os.path.join(train_dir, 'Uninfected')
validation_Parasitized_dir = os.path.join(validation_dir, 'Parasitized')
validation_Uninfected_dir = os.path.join(validation_dir, 'Uninfected')

In [ ]:
train_Parasitized_fnames = os.listdir(train_Parasitized_dir)
print(train_Parasitized_fnames[:10])

train_Uninfected_fnames = os.listdir(train_Uninfected_dir)
train_Uninfected_fnames.sort()
print(train_Uninfected_fnames[:10])

In [ ]:
print('total training Parasitized images:', len(os.listdir(train_Parasitized_dir)))
print('total training Uninfected images:', len(os.listdir(train_Uninfected_dir)))
print('total validation Parasitized images:', len(os.listdir(validation_Parasitized_dir)))
print('total validation Uninfected images:', len(os.listdir(validation_Uninfected_dir)))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_Parasitized_pix = [os.path.join(train_Parasitized_dir, fname) 
                for fname in train_Parasitized_fnames[pic_index-8:pic_index]]
next_Uninfected_pix = [os.path.join(train_Uninfected_dir, fname) 
                for fname in train_Uninfected_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_Parasitized_pix+next_Uninfected_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


# **Training the Model** 🏋️

In this section we will be importing the training algorithm and training our model.



In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
img_input = layers.Input(shape=(150, 150, 3))

x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

In [ ]:
x = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(x)

output = layers.Dense(1, activation='sigmoid')(x)

model = Model(img_input, output)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150), 
        batch_size=20,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=50,
        class_mode='binary')

In [ ]:
train_generator = train_datagen.flow_from_directory(
train_dir,  # This is the source directory for training images
target_size=(150, 150),  # All images will be resized to 150x150

batch_size=50,
class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
validation_dir,
target_size=(150, 150),
batch_size=50,
class_mode='binary')

history = model.fit_generator(
train_generator,
steps_per_epoch=25,
epochs=10,
validation_data=validation_generator,
validation_steps=25,
verbose=2
)

# Results 📈

In this section we produce graphs and illustrations that demonstrate the capabilities of the model.

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, val_acc)
plt.title('Accuracy')

plt.figure()

plt.plot(epochs, val_loss)
plt.title('Loss')
print(val_acc)

In [ ]:
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = Model(img_input, successive_outputs)

# Let's prepare a random input image of a parasitized cell or uninfected cell from the training set.
Parasitized_img_files = [os.path.join(train_Parasitized_dir, f) for f in train_Parasitized_fnames]
Uninfected_img_files = [os.path.join(train_Uninfected_dir, f) for f in train_Uninfected_fnames]
img_path = random.choice(Parasitized_img_files + Uninfected_img_files)

img = load_img(img_path, target_size=(150, 150))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, val_acc)
plt.title('Accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, val_loss)
plt.title('Loss')

In [ ]:
model.save('keras.h5')